In [1]:
import pandas as pd
import numpy as np
import pickle
from collections import Counter

In [2]:
product = pd.read_csv('Instacart/products.csv')
orders = pd.read_csv('Instacart/orders.csv')
orders_products = pd.read_csv('Instacart/order_products__train.csv')
aisles = pd.read_csv('Instacart/aisles.csv')
departments = pd.read_csv('Instacart/departments.csv')
orders_products_prior = pd.read_csv('Instacart/order_products__prior.csv')

In [3]:
product.head(n=3)

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7


In [4]:
orders_products_prior.head(n=2)

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1


In [5]:
'''
Map product IDs to product names.
'''
id_to_name_dict = product[['product_id', 'product_name']].set_index('product_id').to_dict()
id_to_name_dict = id_to_name_dict['product_name']

In [6]:
len(id_to_name_dict)

49688

In [7]:
orders.head(n=2)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0


In [9]:
orders_products.head(n=2)

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1


In [10]:
def create_order_id_to_product_ids(dataset):
    order_id_to_products = {}
    for _, row in dataset.iterrows():
        if row['order_id'] in order_id_to_products:
            order_id_to_products[row['order_id']].append(
                row['product_id']
            )
        else:
            order_id_to_products[row['order_id']] = [
                row['product_id']
            ]
    return order_id_to_products
order_to_products_train = create_order_id_to_product_ids(orders_products)
# order_to_products_prior = create_order_id_to_product_ids(orders_products_prior)

In [11]:
def create_user_id_to_order_ids():     
    user_id_to_order_id = {}
    for _, row in orders.iterrows():
        if len(user_id_to_order_id) >= 5000: break
        if row['user_id'] in user_id_to_order_id:
            user_id_to_order_id[row['user_id']].append(
                row['order_id']
            )
        else:
            user_id_to_order_id[row['user_id']] = [
                 row['order_id']
            ]
    return user_id_to_order_id
user_to_order = create_user_id_to_order_ids()

In [12]:
def create_user_to_products_dict():
    user_to_products_train = {}
    user_to_products_prior = {}
    
    for user in user_to_order:
        cur_user_orders = user_to_order[user]
        for order in cur_user_orders:
            if order in order_to_products_train:
                if user in user_to_products_train:
                    user_to_products_train[user] += order_to_products_train[order]
                else:
                    user_to_products_train[user] = order_to_products_train[order]
            else:
                pass
#                 if user in user_to_products_prior:
#                     user_to_products_prior[user] += order_to_products_prior[order]
#                 else:
#                     user_to_products_prior[user] = order_to_products_prior[order]
                    
    return user_to_products_train, user_to_products_prior

users_to_products_train, users_to_products_prior = create_user_to_products_dict()

In [13]:
print len(users_to_products_train)
# print len(user_to_products_prior)
# print len(order_to_products)
# print len(user_to_order)

3200


In [14]:
# '''
# Generate the dict 
#     user -> Counter({product_id : num_times_purchased})
# '''
# users_to_num_products = {}
# for user in users_to_products_train:
#     users_to_num_products[user] = Counter(users_to_products_train[user])

In [15]:
users_to_binary_product_array = {}
for user in users_to_products_train:
    binary_list = [0]*49688
    for prod_id in users_to_products_train[user]:
        binary_list[prod_id-1] = 1
    users_to_binary_product_array[user] = binary_list

In [16]:
def  save_dict():
    with open('./users_to_products.pkl', 'wb') as f:
        pickle.dump(
            users_to_binary_product_array,
            f, pickle.HIGHEST_PROTOCOL
        )

In [ ]:
save_dict()